In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import json
import time

from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException
from tencentcloud.nlp.v20190408 import nlp_client, models

import jieba
import matplotlib.pyplot as plt

from pyecharts import options as opts
from pyecharts.charts import WordCloud
from pyecharts.charts import Pie
from pyecharts.charts import Bar
from pyecharts.charts import Line
from pyecharts.charts import Scatter

In [ ]:
class word:
    def __init__(self,keyword,SentimentResp):
        self.word=keyword.Word
        self.occurance=1
        self.score=keyword.Score
        self.negative=list()
        self.neutral=list()
        self.positive=list()
        if SentimentResp.Sentiment=="positive":
            self.sentiment=[1,0,0]
            self.positive.append((SentimentResp.Positive,SentimentResp.Neutral,SentimentResp.Negative))
        elif SentimentResp.Sentiment=="neutral":
            self.sentiment=[0,1,0]
            self.neutral.append((SentimentResp.Positive,SentimentResp.Neutral,SentimentResp.Negative))
        else:
            self.sentiment=[0,0,1]
            self.negative.append((SentimentResp.Positive,SentimentResp.Neutral,SentimentResp.Negative))
        
    def add(self,keyword,SentimentResp):
        self.occurance+=1
        self.score+=keyword.Score
        if SentimentResp.Sentiment=="positive":
            self.sentiment[0]+=1
            self.positive.append((SentimentResp.Positive,SentimentResp.Neutral,SentimentResp.Negative))
        elif SentimentResp.Sentiment=="neutral":
            self.sentiment[1]+=1
            self.neutral.append((SentimentResp.Positive,SentimentResp.Neutral,SentimentResp.Negative))
        else:
            self.sentiment[2]+=1
            self.negative.append((SentimentResp.Positive,SentimentResp.Neutral,SentimentResp.Negative))

In [ ]:
class word2:
    def __init__(self,cutword,SentimentResp):
        self.word=cutword
        self.occurance=1
        self.negative=list()
        self.neutral=list()
        self.positive=list()
        if SentimentResp.Sentiment=="positive":
            self.sentiment=[1,0,0]
            self.positive.append((SentimentResp.Positive,SentimentResp.Neutral,SentimentResp.Negative))
        elif SentimentResp.Sentiment=="neutral":
            self.sentiment=[0,1,0]
            self.neutral.append((SentimentResp.Positive,SentimentResp.Neutral,SentimentResp.Negative))
        else:
            self.sentiment=[0,0,1]
            self.negative.append((SentimentResp.Positive,SentimentResp.Neutral,SentimentResp.Negative))
        
    def add(self,SentimentResp):
        self.occurance+=1
        if SentimentResp.Sentiment=="positive":
            self.sentiment[0]+=1
            self.positive.append((SentimentResp.Positive,SentimentResp.Neutral,SentimentResp.Negative))
        elif SentimentResp.Sentiment=="neutral":
            self.sentiment[1]+=1
            self.neutral.append((SentimentResp.Positive,SentimentResp.Neutral,SentimentResp.Negative))
        else:
            self.sentiment[2]+=1
            self.negative.append((SentimentResp.Positive,SentimentResp.Neutral,SentimentResp.Negative))
            
    def getPositiveScore(self):
        if(len(self.positive)==0):
            return 0
        return sum([score[0]-score[2] for score in self.positive])/len(self.positive)
    
    def getNeutralScore(self):
        if(len(self.neutral)==0):
            return 0
        return sum([score[0]-score[2] for score in self.neutral])/len(self.neutral)
    
    def getNegativeScore(self):
        if(len(self.negative)==0):
            return 0
        return sum([score[0]-score[2] for score in self.negative])/len(self.negative)
    
    def getAvgScore(self):
        return (sum([score[0]-score[2] for score in self.positive])+sum([score[0]-score[2] for score in self.neutral])+sum([score[0]-score[2] for score in self.negative]))/(len(self.positive)+len(self.neutral)+len(self.negative))
    
    def getMaxScore(self):
        return max([score[0]-score[2] for score in self.positive]+[score[0]-score[2] for score in self.neutral]+[score[0]-score[2] for score in self.negative])
    
    def getMinScore(self):
        return min([score[0]-score[2] for score in self.positive]+[score[0]-score[2] for score in self.neutral]+[score[0]-score[2] for score in self.negative])

In [ ]:
class dataWrangle:
    def __init__(self):
        file=pd.ExcelFile("【题目2附件】员工反馈表demo.xlsx")
        data=file.parse(0)
        self.feedback=np.append(data.values,data.columns.values)
        self.num=len(self.feedback)
        self.wordDict=dict()
        self.wordDict2=dict()
        self.positive=0
        self.neutral=0
        self.negative=0
        self.sentimentScore=list()
        file.close()
        
        try: 
            cred = credential.Credential("", "") 
            httpProfile = HttpProfile()
            httpProfile.endpoint = "nlp.tencentcloudapi.com"
        
            clientProfile = ClientProfile()
            clientProfile.httpProfile = httpProfile
            client = nlp_client.NlpClient(cred, "ap-guangzhou", clientProfile) 
            
            for sentence in self.feedback:
                if len(sentence)>200:
                    continue
                
                KeywordReq=models.KeywordsExtractionRequest()
                KeywordReq.from_json_string(json.dumps({"Text": sentence}))
                KeywordResp=client.KeywordsExtraction(KeywordReq)
                SentimentReq = models.SentimentAnalysisRequest()
                SentimentReq.from_json_string(json.dumps({"Text": sentence,"Mode":"3class"}))
                SentimentResp=client.SentimentAnalysis(SentimentReq)
                
                if SentimentResp.Sentiment=="positive":
                    self.positive+=1
                elif SentimentResp.Sentiment=="neutral":
                    self.neutral+=1
                else:
                    self.negative+=1
                self.sentimentScore.append(SentimentResp.Positive-SentimentResp.Negative)
                    
                def isChinese(word):
                    for ch in word:
                        if ch < '\u4e00' or ch > '\u9fff':
                            return False
                    return True
                
                for keyword in KeywordResp.Keywords:
                    if len(keyword.Word)>1 and isChinese(keyword.Word):
                        if keyword.Word in self.wordDict:
                            self.wordDict[keyword.Word].add(keyword,SentimentResp)
                        else:
                            self.wordDict[keyword.Word]=word(keyword,SentimentResp)
                
                cutword=jieba.lcut(sentence,cut_all=True) 
                
                for keyword in cutword:
                    if len(keyword)>1 and isChinese(keyword):
                        if keyword in self.wordDict2:
                            self.wordDict2[keyword].add(SentimentResp)
                        else:
                            self.wordDict2[keyword]=word2(keyword,SentimentResp)
                time.sleep(0.1)
                    
           
        except TencentCloudSDKException as err: 
            print(err) 

    
    def getSentiment(self):
        return [self.positive,self.neutral,self.negative]

首先使用pandas读取Excel文件中的反馈，逐条调用腾讯云的情感分析接口和关键词提取接口(需要输入你的腾讯云ID和密钥)，并且将分词后的数据存入字典中，字典的键是单词，值是上面定义的类word，word类能够储存每个词的出现次数和对应的反馈的情感分析结果。

In [ ]:
data=dataWrangle()

In [ ]:
attr = ["正面", "负面", "中性"]
(
    Pie()
    .add("", list(zip(attr, data.getSentiment())))
    .set_global_opts(title_opts=opts.TitleOpts(title="反馈情感分析"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"),
                     tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)")
                    )
    .render_notebook()
)

通过调用dataWrangle类的构造函数读取数据并进行分析后获得了关于264条反馈的情感分析。根据腾讯云nlp情感分析接口分析的结果，其中正面反馈58条,占比21.97%，负面反馈108条，占比40.91%，中性反馈98条，占比37.12%。

In [ ]:
x=[round(-0.9+0.2*i,1) for i in range(10)]
y=[0 for i in range(10)]
for score in data.sentimentScore:
    y[int(score//0.2)+5]+=1
yitem=list()
for i in range(10):
    if i<4:
        yitem.append(
            opts.BarItem(
                value=y[i],
                itemstyle_opts=opts.ItemStyleOpts(color="#2f4554"),
            )
        )
    elif i<6:
        yitem.append(
            opts.BarItem(
                value=y[i],
                itemstyle_opts=opts.ItemStyleOpts(color="#61a0a8"),
            )
        )
    else:
        yitem.append(
            opts.BarItem(
                value=y[i],
                itemstyle_opts=opts.ItemStyleOpts(color="#c23531"),
            )
        )

(
    Bar()
    .add_xaxis(x)
    .add_yaxis("反馈数量", yitem, category_gap=0)
    .set_global_opts(title_opts=opts.TitleOpts(title="反馈得分分布"),tooltip_opts=opts.TooltipOpts(is_show=False))
    .render_notebook()
)

腾讯云会对情感分析结果返回的具体分数，包括Positive, Neutral和Negative三项分数，总和为1。在这里我定义情感得分为Positive-Negative，取值范围为[-1,1]，并制作直方图。可以看到按0.2分一个区间区分时反馈最多的区间是[-0.2,0]和[0,0.2]，这是因为中性反馈基本都落在了这两个区间，而数量更多的负面反馈分布在四个区间里。

In [ ]:
(
    WordCloud()
    .add(series_name="关键词词云图", data_pair=[(i,data.wordDict[i].score) for i in data.wordDict], word_size_range=[10, 100])
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="关键词词云图", title_textstyle_opts=opts.TextStyleOpts(font_size=23)
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
    .render_notebook()
)

根据腾讯云nlp的关键词提取接口获得了各个词的关键词得分并生成词云图，可以发现腾讯云对关键词的切割不够彻底，在图中我们可以发现“管理”和“管理层”、“内部管理”等词同时出现，关键词统计到的次数也少于预期，出现最多的关键词“公司”和“员工”仅有25分左右，说明最多统计到了不到26次。

In [ ]:
(
    WordCloud()
    .add(series_name="关键词词云图2", data_pair=[(i,data.wordDict2[i].occurance) for i in data.wordDict2], word_size_range=[15, 100])
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="关键词词云图2", title_textstyle_opts=opts.TextStyleOpts(font_size=23)
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
    .render_notebook()
)

因此我使用了另一个中文分词库jieba对每条反馈进行分词，并且在分词时使用全模式，可以将单词切割的更加精细，在新生成的词云图上可以看到关键词的统计次数都有明显增加。但是也有一些无用的词出现，比如连接词，语气词以及错误切割的词。

In [ ]:
top10word=sorted(data.wordDict2.keys(),key=lambda x:data.wordDict2[x].occurance,reverse=True)[0:10]
(
    Bar()
    .add_xaxis(top10word)
    .add_yaxis("负面", [data.wordDict2[i].sentiment[2] for i in top10word], stack="stack1",
              itemstyle_opts=opts.ItemStyleOpts(color="#2f4554"))
    .add_yaxis("中性", [data.wordDict2[i].sentiment[1] for i in top10word], stack="stack1",
              itemstyle_opts=opts.ItemStyleOpts(color="#61a0a8"))
    .add_yaxis("正面", [data.wordDict2[i].sentiment[0] for i in top10word], stack="stack1",
              itemstyle_opts=opts.ItemStyleOpts(color="#c23531"))
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title="关键词情感分布"))
    .render_notebook()
)

接下来对出现较多的关键词进行分析，因为jieba能够更好的统计到出现的较多的关键词数量，而jieba分出的无用词出现次数都会太多，所以采用jieba分词产生的关键词进行分析。出现次数前十的词是['管理', '部门', '公司', '业务', '员工', '没有', '工作', '人员', '不够', '问题']，可以看到出现次数前十的词中大部分都以负面和中性评价为主，符合所有反馈的总体趋势。值得注意的是唯二的正面情感多于负面情感的词是['没有', '问题']，这两个词反而是词语本身含义比较负面的，可以猜想或许是几个负面的词结合表达出了正面含义，比如“没有问题”。在未来也可以研究不同词之间出现的关系。

In [ ]:
line=(
    Line()
    .add_xaxis(top10word)
    .add_yaxis("正面情感分数", [data.wordDict2[i].getPositiveScore() for i in top10word])
    .add_yaxis("负面情感分数", [data.wordDict2[i].getNegativeScore() for i in top10word])
    .add_yaxis("中性情感分数", [data.wordDict2[i].getNeutralScore() for i in top10word])
    .add_yaxis("平均情感分数", [data.wordDict2[i].getAvgScore() for i in top10word])
    .set_global_opts(title_opts=opts.TitleOpts(title="关键词情感分数"))
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
scatter = (
    Scatter()
    .add_xaxis(top10word)
    .add_yaxis("最正面分数", [round(data.wordDict2[i].getMaxScore(),2) for i in top10word],
               itemstyle_opts=opts.ItemStyleOpts(color="#c23531")
              )
    .add_yaxis("最负面分数", [round(data.wordDict2[i].getMinScore(),2) for i in top10word],
               itemstyle_opts=opts.ItemStyleOpts(color="#2f4554"))
)
line.overlap(scatter).render_notebook()

对关键词的具体情感分数进行分析，将正面、负面、中性的评价分别统计分数，可以看到部门是负面反馈中负面情绪最重的关键词，而人员是正面反馈中正面情绪最重的关键词。另外每个关键词最正面和最负面的反馈对于关键词的平均情感分数影响不大，主要影响因素还是各个情绪上的反馈数量。